In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-07-13 23:13:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-07-13 23:13:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-07-13 23:13:17--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [5]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [6]:
%ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   sample_data/


In [7]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file='glove.6B.200d.txt'
word2vec_output_file='word2vec.txt'
glove2word2vec(glove_input_file,word2vec_output_file)

<ipython-input-7-62d0641268e2>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file,word2vec_output_file)


(400000, 200)

In [8]:
from gensim.models import KeyedVectors
model= KeyedVectors.load_word2vec_format(word2vec_output_file)

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

# Load the dataset
data = pd.read_csv('dataset.csv')

# Preprocess the data
texts = data['Text'].values
labels = data['Label'].values

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Tokenize the text
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=200)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [11]:
def glove_embeddings(filepath, word_index, embedding_dim):
    embeddings_index = {}
    with open(filepath, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_dim = 200
glove_filepath = 'glove.6B.200d.txt'
embedding_matrix = glove_embeddings(glove_filepath, tokenizer.word_index, embedding_dim)

In [14]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Activation
# Build the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=200))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.4))
model.add(Dense(256,activation='sigmoid'))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64,callbacks=[early_stopping])

Epoch 1/10
220/220 [==============================] - 1997s 9s/step - loss: 0.6414 - accuracy: 0.6252
Epoch 2/10
220/220 [==============================] - 1991s 9s/step - loss: 0.5572 - accuracy: 0.7111
Epoch 3/10
220/220 [==============================] - 2005s 9s/step - loss: 0.4971 - accuracy: 0.7594
Epoch 4/10
220/220 [==============================] - 1998s 9s/step - loss: 0.4208 - accuracy: 0.8095
Epoch 5/10
220/220 [==============================] - 1997s 9s/step - loss: 0.3418 - accuracy: 0.8553
Epoch 6/10
220/220 [==============================] - 1995s 9s/step - loss: 0.2690 - accuracy: 0.8927
Epoch 7/10
220/220 [==============================] - 1999s 9s/step - loss: 0.2053 - accuracy: 0.9214
Epoch 8/10
220/220 [==============================] - 1944s 9s/step - loss: 0.1600 - accuracy: 0.9429
Epoch 9/10
220/220 [==============================] - 1951s 9s/step - loss: 0.1324 - accuracy: 0.9562
Epoch 10/10
220/220 [==============================] - 1946s 9s/step - loss: 0.107

In [15]:
from sklearn.metrics import classification_report
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")
y_pred = y_pred.flatten()
print(classification_report(y_test,y_pred))

110/110 [==============================] - 177s 2s/step
              precision    recall  f1-score   support

           0       0.74      0.74      0.74      2094
           1       0.62      0.62      0.62      1426

    accuracy                           0.69      3520
   macro avg       0.68      0.68      0.68      3520
weighted avg       0.69      0.69      0.69      3520



In [16]:
from sklearn.metrics import roc_auc_score

roc_auc=roc_auc_score(y_test,y_pred)
print('ROC-AUC SCORE:',roc_auc)

ROC-AUC SCORE: 0.6780710532061818
